# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask.dataframe as dd

c:\Users\sahil\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [19]:
import os
import sys
from glob import glob

# Write your code below.
sys.path.append(os.getenv('SRC_DIR'))
from utils.logger import get_logger
_logs = get_logger(__name__)

price_data_dir = os.getenv('PRICE_DATA')

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)

if not parquet_files:
    _logs.error(f"No Parquet files found in {price_data_dir}")

_logs.info(f"Found {len(parquet_files)} parquet files in {price_data_dir}")

2025-06-13 16:08:03,444, 2674667290.py, 17, INFO, Found 2889 parquet files in ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
# Write your code below.

ddf = dd.read_parquet(parquet_files)

ddf = ddf.sort_values(by=['ticker', 'Date'])

ddf['Close_lag_1'] = ddf.groupby('ticker')['Close'].shift(1)
ddf['Adj_close_lag_1'] = ddf.groupby('ticker')['Adj Close'].shift(1)

ddf['returns'] = (ddf['Close'] / ddf['Close_lag_1']) - 1

ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

dd_feat = ddf

dd_feat

C:\Users\sahil\AppData\Local\Temp\ipykernel_13100\3595039699.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Close_lag_1'] = ddf.groupby('ticker')['Close'].shift(1)
C:\Users\sahil\AppData\Local\Temp\ipykernel_13100\3595039699.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf['Adj_close_lag_1'] = ddf.groupby('ticker')['Adj Close'].shift(1)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_close_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
dd_feat.compute()

ValueError: cannot reindex on an axis with duplicate labels

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.
df_feat = dd_feat.compute()
df_feat = df_feat.sort_values(by=['ticker', 'Date'])
df_feat = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())


ValueError: cannot reindex on an axis with duplicate labels

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Pandas is great for calculating the moving average return especially in a smaller dataset.
Dask wouldn't be requried in this case as this isn't a large dataset. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.